## オブザーバビリティを高める機能

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

実行に必要な環境変数を読み込みます。

In [2]:
import os, warnings, time

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://localhost:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

### プロンプト

各トレースを同一グループにするために、セッションIDをUUID v4ベースで生成する

In [3]:
from langfuse import Langfuse

# Initialize Langfuse client
langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
    )


プロンプトテンプレートの作成

In [4]:
# Create a text prompt
langfuse.create_prompt(
    name="test-sdk",
    type="text",
    prompt="{{topic}}について教えて",
    labels=["sdk"],
    config={},
)

In [34]:
langfuse.create_prompt(
    name="test-sdk",
    type="text",
    prompt="簡潔に{{topic}}について教えて",
    labels=["sdk","production"],
    config={},
)

In [38]:

# プロンプトテンプレート`test-sdk"`を取得
langfuse_prompt = langfuse.get_prompt("test-sdk")

# 取得したプロンプトから
compiled_prompt = langfuse_prompt.compile(topic="大爆笑一発ギャグ")
# -> "As an expert movie critic, do you like Dune 2?"
compiled_prompt

'簡潔に大爆笑一発ギャグについて教えて'

In [53]:
import uuid

session_id = str(uuid.uuid4())

from langfuse.decorators import langfuse_context, observe

from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere
from langfuse.callback import CallbackHandler

callback_handler = CallbackHandler(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint,
    session_id=session_id,
)
#@observe#(as_type="generation")

@observe()
def nested_generation():
    langchain_prompt = ChatPromptTemplate.from_template(
        langfuse_prompt.get_langchain_prompt(),
        metadata={"langfuse_prompt": langfuse_prompt},
    )
    langfuse_context.update_current_observation(
        prompt=langchain_prompt,
    )
    model = ChatCohere(
        model="command-r-plus",
        temperature=1,
    )
    chain = langchain_prompt | model
    chain.invoke(
        input={"topic": "大爆笑一発ギャグ"},
        config={"callbacks": [callback_handler]},
    )

    #return langchain_prompt

nested_generation()

validate_env
generate
{'message': '簡潔に大爆笑一発ギャグについて教えて', 'chat_history': [], 'model': 'command-r-plus', 'temperature': 1.0}
basecohere.chat
text='一発ギャグとは、1つのフレーズや動作で笑いを取るギャグのことです。代表的なものとしては、「コマネチ」（体操選手のコマネチの真似をする）、「いいんです！」（両手を広げて「いいんです！」と叫ぶ）、「つけ麺、ダクダク」（「つけ麺」と言いながら涙を流す仕草をする）などがあります。一発ギャグは、タイミングや動き、声のトーンなどが重要で、笑いを取るのはなかなか難しいですが、上手く決まるととても盛り上がります。' generation_id='0dbed4e0-590e-4196-ac29-2dea0ac760d4' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[Message_User(message='簡潔に大爆笑一発ギャグについて教えて', tool_calls=None, role='USER'), Message_Chatbot(message='一発ギャグとは、1つのフレーズや動作で笑いを取るギャグのことです。代表的なものとしては、「コマネチ」（体操選手のコマネチの真似をする）、「いいんです！」（両手を広げて「いいんです！」と叫ぶ）、「つけ麺、ダクダク」（「つけ麺」と言いながら涙を流す仕草をする）などがあります。一発ギャグは、タイミングや動き、声のトーンなどが重要で、笑いを取るのはなかなか難しいですが、上手く決まるととても盛り上がります。', tool_calls=None, role='CHATBOT')] prompt=None meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), bill

In [5]:
langfuse.create_dataset(
    name="test",
    # optional description
    description="My first dataset",
    # optional metadata
    metadata={
        "author": "Alice",
        "date": "2022-01-01",
        "type": "benchmark"
    }
)

Dataset(id='cm2ki8l2g001bu5nhzgiebjij', name='test', description='My first dataset', metadata={'date': '2022-01-01', 'type': 'benchmark', 'author': 'Alice'}, project_id='pj-1234567890', created_at=datetime.datetime(2024, 10, 22, 13, 52, 11, 945000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 10, 22, 15, 58, 2, 885000, tzinfo=datetime.timezone.utc))

In [7]:
langfuse.create_dataset_item(
    dataset_name="test",
    # any python object or value, optional
    input={
        "text": "hello world"
    },
    # any python object or value, optional
    expected_output={
        "text": "hello world"
    },
    # metadata, optional
    metadata={
        "model": "llama3",
    }
)

DatasetItem(id='a54cadf3-ec66-49bd-aff2-46413c1aeeb9', status=<DatasetStatus.ACTIVE: 'ACTIVE'>, input={'text': 'hello world'}, expected_output={'text': 'hello world'}, metadata={'model': 'llama3'}, source_trace_id=None, source_observation_id=None, dataset_id='cm2ki8l2g001bu5nhzgiebjij', dataset_name='test', created_at=datetime.datetime(2024, 10, 22, 15, 59, 11, 538000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 10, 22, 15, 59, 11, 538000, tzinfo=datetime.timezone.utc))

In [8]:
langfuse.create_dataset_item(
    dataset_name="test",
    input={ "text": "hello world" },
    expected_output={ "text": "hello world" },
    # link to a trace
    source_trace_id="b35c6305-b639-4670-a755-bfd5e725791f",
    # optional: link to a specific span, event, or generation
    #source_observation_id="<observation_id>"
)

DatasetItem(id='247bb60a-d700-47ef-8d52-0bd01f8e2fe2', status=<DatasetStatus.ACTIVE: 'ACTIVE'>, input={'text': 'hello world'}, expected_output={'text': 'hello world'}, metadata=None, source_trace_id='b35c6305-b639-4670-a755-bfd5e725791f', source_observation_id=None, dataset_id='cm2ki8l2g001bu5nhzgiebjij', dataset_name='test', created_at=datetime.datetime(2024, 10, 22, 16, 2, 22, 455000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 10, 22, 16, 2, 22, 455000, tzinfo=datetime.timezone.utc))

In [11]:
from langfuse.decorators import observe

@observe()
def hello_world(data: str):
    print("Hello " + data)

In [29]:
langfuse.create_dataset(name="japanese_holidays")
local_items = [
    {"input": {"year": "2024","name":"海の日"}, "expected_output": "2024/07/15"},
    {"input": {"year": "2024","name":"天皇誕生日"}, "expected_output": "2024/02/23"},
    {"input": {"year": "2024","name":"勤労感謝の日"}, "expected_output": "2024/11/23"},
]
for item in local_items:
  langfuse.create_dataset_item(
      dataset_name="japanese_holidays",
      input=item["input"],
      expected_output=item["expected_output"]
)

In [42]:
from langchain_cohere import ChatCohere

from langchain_core.prompts import ChatPromptTemplate

def run_my_langchain_llm_app(input, callback_handler):
  prompt = ChatPromptTemplate.from_messages(
    [
        ("system","あなたは日本の祝日に詳しい有能なアシスタントです。"),
        ("user","{year}年の{name}をYYYY/MM/DD形式で回答してください。例えば、元日であれば「2024/01/01」と回答してください。"),
    ]
  )
  chat = ChatCohere(
    model="command-r-plus",
    temperature=0,
  )

  chain = prompt | chat

  res = chain.invoke(
    input,
    config={"callbacks":[callback_handler]}
  )

  return res

In [43]:
dataset = langfuse.get_dataset("japanese_holidays")

for item in dataset.items:
  handler = item.get_langchain_handler(run_name="case1")

  completion = run_my_langchain_llm_app(item.input, handler)

  exact_match = lambda str1, str2: str1 == str2
  handler.trace.score(
    name="exact_match",
    data_type="BOOLEAN",
    value= exact_match(completion.content, item.expected_output)
  )

validate_env
generate
get_role
{'message': '2024年の勤労感謝の日をYYYY/MM/DD形式で回答してください。例えば、元日であれば「2024/01/01」と回答してください。', 'chat_history': [{'role': 'System', 'message': 'あなたは日本の祝日に詳しい有能なアシスタントです。'}], 'model': 'command-r-plus', 'temperature': 1.0}
basecohere.chat
text='2024/11/23' generation_id='4f18355e-342b-42a3-a42d-be0a20fe4eea' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[Message_System(message='あなたは日本の祝日に詳しい有能なアシスタントです。', tool_calls=None, role='SYSTEM'), Message_User(message='2024年の勤労感謝の日をYYYY/MM/DD形式で回答してください。例えば、元日であれば「2024/01/01」と回答してください。', tool_calls=None, role='USER'), Message_Chatbot(message='2024/11/23', tool_calls=None, role='CHATBOT')] prompt=None meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=59.0, output_tokens=10.0, search_units=None, classifications=None), tokens=ApiMetaTokens(